In [1]:
import anthropic
import time
from tqdm import tqdm
import os
import re
from collections import deque
from initialize import *
%load_ext autoreload
%autoreload 2

api_key = os.environ.get("ANTHROPIC_API_KEY_SPAR")#os.environ.get("ANTHROPIC_API_KEY_SHARED")#os.environ.get("ANTHROPIC_API_KEY")
client = anthropic.Anthropic(api_key=api_key)
tokqpm = 50

/Users/christopherackerman/repos/spar_self_recognition/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print("API key:",api_key)

In [35]:
### Create writing recognition judgment prompts using articles and summaries and appropriate formatting

def trim_to_set_length(text, target_length=1200):
    # Find the last word boundary before or just after the target length
    match = re.search(r'\s+\S+\s*$', text[:min(target_length+4,len(text))])
    if match:
        trim_index = match.start()
    else:
        # If no word boundary found, just cut at target length
        trim_index = target_length
    
    return text[:trim_index]
    
def trim_to_same_length(text1, text2):
    if abs(len(text1) - len(text2)) < 5: return text1, text2 #close enough
    if max(len(text1), len(text2)) < 30: return text1, text2 #too short to trim

    # Determine which text is longer
    if len(text1) > len(text2):
        longer_text, shorter_text = text1, text2
    else:
        longer_text, shorter_text = text2, text1
    
    target_length = len(shorter_text)
    if len(shorter_text) > 15: #also truncate the shorter text, so it ends abruptly too
        target_length -= 10
        match = re.search(r'\s+\S+\s*$', shorter_text[:target_length+4])
        if match and match.start() > 0:
            trim_index = match.start()
        else:
            # If no word boundary found, just cut at target length
            trim_index = target_length
        
        shorter_text = shorter_text[:trim_index]
    
    # Find the last word boundary before or just after the target length
    match = re.search(r'\s+\S+\s*$', longer_text[:target_length+4])
    if match and match.start() > 0:
        trim_index = match.start()
    else:
        # If no word boundary found, just cut at target length
        trim_index = target_length
    
    trimmed_text = longer_text[:trim_index]
    
    if len(text1) > len(text2):
        return trimmed_text, shorter_text
    else:
        return shorter_text, trimmed_text
        return text1, trimmed_text
    
def generate_llama_choice_prompts(task, model_name, dataset, fewshot = False, comp_models = [], optlabel = "", alias = ""):
    if dataset == "dolly":
        responses, articles, instructions, keys = load_data_dolly()
    elif dataset == "sad":
        ddirsuffix = "_full" if "full" in optlabel else "_longer" if "longer" in optlabel else ""
        responses, keys = load_data_sad(ddir = f"completions{ddirsuffix}")
    else:
        responses, articles, keys = load_data(dataset)

    prefix = task.value
    questions = []
    if task == TaskType.Individual or task == TaskType.Individual_Continuation:
        suffix = "\n"
        user_prompt = CONTINUATION_RECOGNITION_PROMPT_TEMPLATE if task == TaskType.Individual_Continuation else RECOGNITION_PROMPT_TEMPLATE_ALT
        for key in keys:
            article = articles[key] if task != TaskType.Individual_Continuation else ""
            instruction = instructions[key] if dataset == "dolly" else SUMMARIZE_PROMPT_TEMPLATE_CNN.split('\n\n')[1] if "cnn" in dataset else SUMMARIZE_PROMPT_TEMPLATE_XSUM.split('\n\n')[1] if "xsum" in dataset else ""
            self_response = responses[model_name][key].replace("\n\n","\n").strip()
            if task == TaskType.Individual_Continuation and "full" not in optlabel : self_response = trim_to_set_length(self_response).strip()
            question = {"key": key, "model": model_name}
            if fewshot: 
                question["individual_recognition_prompt"] = make_fewshot_individual_detection_prompt(model_name, article, self_response, instruction, suffix)
            else:
                question["individual_recognition_prompt"] = user_prompt.format(article=article, summary=self_response, inst=instruction)
            questions.append(question)    
            for other in responses.keys():
                if other == model_name: continue
                if len(comp_models) > 0 and other not in comp_models: continue
                question = {"key": key, "model": other}
                other_response = responses[other][key].strip()
                if task == TaskType.Individual_Continuation: other_response = trim_to_set_length(other_response).strip()
                if fewshot: 
                    question["individual_recognition_prompt"] = make_fewshot_individual_detection_prompt(model_name, article, other_response, instruction, suffix)+suffix
                else:
                    question["individual_recognition_prompt"] = user_prompt.format(article=article, summary=other_response, inst=instruction)
                questions.append(question)
    else:
        system_prompt = DETECTION_SYSTEM_PROMPT2###"" if task == TaskType.SelfVOther_Continuation else DETECTION_SYSTEM_PROMPT2
        prefix += "_paired"
        if task == TaskType.SelfVOther_Continuation:
            suffix = "My answer is "####\n\n"
            detect_prompt = CONTINUATION_DETECTION_PROMPT_TEMPLATE###_EXP###_PAPER
            _model_name = model_name if alias == "" else alias
        else:
            if task == TaskType.HumanVModel or task == TaskType.HumanVModel_Continuation: 
                prefix += "_Mis" + "_".join(comp_models)
                _model_name = "human" 
                if "filteredlen" in optlabel: _model_name += "_filteredlen"
            else: _model_name = model_name if alias == "" else alias
            suffix = "My answer is "#"\n"
            ####if task == TaskType.HumanVModel_Continuation: suffix = "\n\n"
            detect_prompt = DETECTION_PROMPT_TEMPLATE_INST if task == TaskType.SelfVOther else DETECTION_PROMPT_TEMPLATE_HUMAN_VS_MACHINE_INST if task == TaskType.HumanVModel else CONTINUATION_DETECTION_PROMPT_TEMPLATE_HUMAN_VS_MACHINE 
        for key in keys:
            if not key in responses[_model_name]: continue
            article = articles[key] if task != TaskType.SelfVOther_Continuation and task != TaskType.HumanVModel_Continuation else ""
            instruction = instructions[key] if dataset == "dolly" else SUMMARIZE_PROMPT_TEMPLATE_CNN.split('\n\n')[1] if "cnn" in dataset else SUMMARIZE_PROMPT_TEMPLATE_XSUM.split('\n\n')[1] if "xsum" in dataset else ""
            self_response = responses[_model_name][key].replace("\n\n","\n").strip()
            for other in responses.keys():
                if other == _model_name: continue
                if len(comp_models) > 0 and other not in comp_models: continue
                if "filteredlen" in other and "filteredlen" not in optlabel: continue
                question = {"key": key, "model": other}
                other_response = responses[other][key].replace("\n\n","\n").strip()
                ###if task == TaskType.SelfVOther_Continuation and "full" not in optlabel: self_response, other_response = trim_to_same_length(self_response, other_response)
                if "full" not in optlabel and "filtered" not in optlabel and "untrimmed" not in optlabel: self_response, other_response = trim_to_same_length(self_response, other_response)
                if fewshot: 
                    question["forward_detection_prompt"] = make_fewshot_pairwise_detection_prompt(model_name, article, self_response, other_response, instruction, suffix, task)+suffix
                    question["backward_detection_prompt"] = make_fewshot_pairwise_detection_prompt(model_name, article, other_response, self_response, instruction, suffix, task)+suffix
                else: 
                    question["forward_detection_prompt"] = detect_prompt.format(article=article, summary1=self_response, summary2=other_response, inst=instruction)
                    question["backward_detection_prompt"] =detect_prompt.format(article=article, summary1=other_response, summary2=self_response, inst=instruction)
                questions.append(question)

    file_name=f"{model_name}_{dataset}_{prefix}{alias}_choice_{optlabel}prompts_sample.json"
    save_to_json(questions[:10], file_name)
    return questions

alias = ""
task = TaskType.HumanVModel#_Continuation#
model_name = "sonnet"
fewshot = False#True if "base" in model_name else False
comp_models = ["llama3_8bchat"]#["llama3_8bchat_filteredlen"]#["human", "claude", "gpt35", "gpt4", "llama", "llama2_13bchat", "llama3_8bbase", "llama3_8bchat"]
datasets = ["xsum", "cnn", "dolly"]#["sad"]#
optlabel = "trim_"#"filteredlen_"#
prompts_dict = {}
for dataset in datasets:
    prompts = generate_llama_choice_prompts(task = task, model_name = model_name, dataset = dataset, fewshot = fewshot, comp_models = comp_models, optlabel = optlabel, alias = alias)
    prompts_dict[dataset] = prompts

In [45]:
# Get Anthropic model judgments of article summaries, given appropriate prompts; writes to choice_results/..._pairwise_untuned.json or individual_results/..._individual_untuned.json, as appropriate

def compute_choice_results(model_name, questions, ds_name, task = TaskType.SelfVOther, optlabel = "", alias = ""):
    """
    Given prompts asking for a binary choice between two options (summary_id or yes/no), generates the model's choice and log probabilities
    and writes to resultsdir/..._task_untuned.json.
    """
    suffix = "My answer is " #"\n"#""##Starting the assistant's response like that really does help, both reducing ambigious responses and accuracy -- actually it turns out that just \n is enough
    prefix=""
    textlabel = "Summary"
    if task == TaskType.SelfVOther:
        dir_name = "choice_results"
    elif task == TaskType.HumanVModel:
        dir_name = "choice_results_humanVmodel"
        prefix = "_Mis" + "_".join(comp_models)
    elif task == TaskType.SelfVOther_Continuation or task == TaskType.Individual_Continuation or task == TaskType.HumanVModel_Continuation:
        dir_name = "completions_longer"
        textlabel = "Text"
    else:
        dir_name = "individual_results"
#        suffix = "\n"
    
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    
    outputfile = f"{ds_name}_{model_name}_{task.value}{alias}{prefix}_{optlabel}untuned"
    filename = f"{dir_name}/{outputfile}.json"
    partial_filename = f"{dir_name}/{outputfile}_partial.json"
    
    results = []    
    if os.path.exists(filename) or os.path.exists(partial_filename):
        results = load_from_json(filename) if os.path.exists(filename) else load_from_json(partial_filename)
        existing_combinations = {(result['key'], result['model']) for result in results}
        questions = [q for q in questions if (q['key'], q['model']) not in existing_combinations]
        if questions == []:
            print(f"All models in {filename} have been processed. Exiting.")
            return
        else:
            print(f"Processing new data not in {filename}")
    initlenresults = len(results)
    subtasks = ["forward_detection", "backward_detection"] if task != TaskType.Individual and task != TaskType.Individual_Continuation else ["individual_recognition"]
    if task == TaskType.Individual or task == TaskType.Individual_Continuation: tokens = ["Yes", "No"]
    elif task == TaskType.SelfVOther_Continuation: tokens = ["1", "2"]###["A", "B"]
    else: tokens = ["1", "2"]
    batch_size = 1

    start_time = time.time()
    queue = deque(maxlen=tokqpm)
    for i in tqdm(range(initlenresults, initlenresults+len(questions), batch_size)):
        #print(f"Processing batch {i} to {i + batch_size}")
        batch = questions[i-initlenresults:i-initlenresults+batch_size]
        question = batch[0]
        
        result = {"key": question["key"], "model": question["model"]}
        for subtask in subtasks:
            input_texts = [item[f"{subtask}_prompt"] + suffix for item in batch] 
            
            for attempt in range(12):
                if len(queue) == tokqpm:
                    time_since_first_request = time.time() - queue[0]
                    if time_since_first_request < 60:  # If less than a minute has passed
                        time.sleep(60 - time_since_first_request)  # Sleep the remaining seconds
                try:
                    message = client.messages.create(
                        model="claude-3-5-sonnet-20240620",
                        max_tokens=2,
                        temperature=0.0+attempt*.09,
                        system=RECOGNITION_SYSTEM_PROMPT2 if task == "individual" else DETECTION_SYSTEM_PROMPT2,
                        messages=[
                            {"role": "user", "content": input_texts[0]}
                            #,{"role": "assistant", "content": "My answer is"}
                        ]
                    )
                    queue.append(time.time())
                    resp = message.content[0].text.strip()
                    if resp not in tokens: continue
                    break
                except Exception as e:
                    print(f"Error: {e}")
                    if attempt == 11:
                        print("Failed to get response. Stopping.")
                        exit(1)
                    time.sleep(5)

            if resp not in tokens:
                print(f"Final invalid response at id {question['key']}: {resp}")

            result[subtask] = resp
            result[f"{subtask}_logprob"] = {tokens[0]: .5, tokens[1]: .5}
            for j, item in enumerate(batch):
                if subtask == subtasks[0]: # no reason to do this more than once:
                    if task != TaskType.Individual and task != TaskType.Individual_Continuation and not fewshot:
                        result["self_summary_perplexity"] = 1
                        result["other_summary_perplexity"] = 1
                    elif not fewshot:
                        result["self_summary_perplexity"] = 1
        
        results.append(result)
            
        
        if (i-initlenresults + batch_size) % 100 == 0 or (i-initlenresults + batch_size) >= len(questions):
            print(f"Completed {i-initlenresults + batch_size} rows out of {len(questions)}")
            save_to_json(results, partial_filename)

    save_to_json(results, filename)

for dataset in prompts_dict.keys():
    compute_choice_results(model_name, prompts_dict[dataset], dataset, task=task, optlabel=optlabel, alias = alias)


Processing new data not in choice_results_humanVmodel/xsum_sonnet_HvM_Misllama3_8bchat_trim_untuned.json


  0%|          | 0/28 [00:00<?, ?it/s]

Error: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}
Error: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}


  0%|          | 0/28 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [36]:
prompts_dict[dataset][:2]

[{'key': 'id0',
  'model': 'llama3_8bchat',
  'forward_detection_prompt': 'I am going to show you a news article, followed by two summaries of it, in random order. One of the summaries was generated by an AI model, and one was written by a human, and I want you to tell me which one you think an AI model wrote. The prompt given to the summary writer was: "What is a dispersive prism?". Think about writing styles and the kinds of things people and AI models would say about this article given that prompt, and based on that identify the summary you think an AI model wrote by outputting its corresponding number, either "1" or "2".\n\nArticle:\nIn optics, a dispersive prism is an optical prism that is used to disperse light, that is, to separate light into its spectral components (the colors of the rainbow). Different wavelengths (colors) of light will be deflected by the prism at different angles. This is a result of the prism material\'s index of refraction varying with wavelength (dispersi

In [44]:
### clean file
file_path = 'choice_results_humanVmodel/xsum_sonnet_HvM_Misllama3_8bchat_trim_untuned.json'

with open(file_path, 'r') as file:
    data = json.load(file)

filtered_data = [
    entry for entry in data
    if entry['forward_detection'] in ('1', '2') and entry['backward_detection'] in ('1', '2')
]

with open(file_path, 'w') as file:
    json.dump(filtered_data, file)


In [ ]:
### Call Sonnet to assemble list of ways human and AI-generated texts differ, using the text pairs that Llama3-chat could easily distinguish in individual recognition paradigm
prompt_template = """I want your help in distinguishing the writings of two different authors. I have several samples of each of their writings. I will show you the writings in the format: 

Author 1, Text1:
[text]

Author 2, Text1:
[text]

Author 1, Text2:
[text]

Author 2, Text2:
[text]

etc. I want you to closely exampine the writing style, format, tone, and content of each, and, generalizing across the samples, identify two to four ways in which the authors' writing can be distinguised. I am looking for general rules/cahracterstics I can then apply to other texts to figure out which author likely wrote them. Here are the samples:

Author 1, Text1:
{a1t1}

Author 2, Text1:
{a2t1}

Author 1, Text2:
{a1t2}

Author 2, Text2:
{a2t2}

Author 1, Text3:
{a1t3}

Author 2, Text3:
{a2t3}

Author 1, Text4:
{a1t4}

Author 2, Text4:
{a2t4}

Author 1, Text5:
{a1t5}

Author 2, Text5:
{a2t5}
"""

characteristics = []
df = pd.read_csv("./llama3_8bchat_individual_strong_predictions_new.csv")
start_time = time.time()
queue = deque(maxlen=tokqpm)

do three passes
    reorder randomly dataset
    while not at end of dataset:
        read in next five pairs
        model_text, human_text = row['self_output'], row['other_output']
        alternate between model being author 1 and 2
        fill prompt_template and call claude

                
        for attempt in range(12):
            if len(queue) == tokqpm:
                time_since_first_request = time.time() - queue[0]
                if time_since_first_request < 60:  # If less than a minute has passed
                    time.sleep(60 - time_since_first_request)  # Sleep the remaining seconds
            try:
                message = client.messages.create(
                    model="claude-3-5-sonnet-20240620",
                    max_tokens=1024,
                    temperature=0.0+attempt*.09,
                    messages=[
                        {"role": "user", "content": input_text}
                    ]
                )
                queue.append(time.time())
                resp = message.content[0].text.strip()
                break
            except Exception as e:
                print(f"Error: {e}")
                if attempt == 11:
                    print("Failed to get response. Stopping.")
                    exit(1)
                time.sleep(5)

        parse its output into a list and append that to characteristics

dedup


In [ ]:
import pandas as pd

MAX_ATTEMPTS = 12

df = pd.read_csv("./llama3_8bchat_individual_strong_predictions_new.csv")
start_time = time.time()
queue = deque(maxlen=tokqpm)

def process_text(input_text):
    prompt = f"I want you to read the following text and tell me whether it has any person names in it. Respond only with the word \"Yes\", if it does include at least one person's name, or \"No\" if it doesn't. Text:\n{input_text}"
    
    for attempt in range(MAX_ATTEMPTS):
        if len(queue) == tokqpm:
            time_since_first_request = time.time() - queue[0]
            if time_since_first_request < 60:  # If less than a minute has passed
                time.sleep(60 - time_since_first_request)  # Sleep the remaining seconds
        
        try:
            message = client.messages.create(
                model="claude-3-5-sonnet-20240620",
                max_tokens=1,
                temperature=0.0 + attempt * 0.09,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )
            queue.append(time.time())
            resp = message.content[0].text.strip().lower()
            
            if resp in ["yes", "no"]:
                return resp
        except Exception as e:
            print(f"Error: {e}")
        
        time.sleep(5)
    
    print(f"Failed to get valid response for text: {input_text[:50]}...")
    return None

# Process each row in the dataframe
for index, row in df.iterrows():
    model_text, human_text = row['self_output'], row['other_output']
    
    model_resp = process_text(model_text)
    human_resp = process_text(human_text)
    
    df.at[index, 'model_text_has_name'] = model_resp
    df.at[index, 'human_text_has_name'] = human_resp

# Trim the dataframe to have equal numbers of texts with and without names
min_count = min(
    (df['model_text_has_name'] == 'yes').sum(),
    (df['model_text_has_name'] == 'no').sum(),
    (df['human_text_has_name'] == 'yes').sum(),
    (df['human_text_has_name'] == 'no').sum()
)

model_with_name = df[df['model_text_has_name'] == 'yes'].sample(min_count)
model_without_name = df[df['model_text_has_name'] == 'no'].sample(min_count)
human_with_name = df[df['human_text_has_name'] == 'yes'].sample(min_count)
human_without_name = df[df['human_text_has_name'] == 'no'].sample(min_count)

trimmed_df = pd.concat([model_with_name, model_without_name, human_with_name, human_without_name])

# Write the results to a new CSV file
trimmed_df.to_csv("./llama3_8bchat_individual_strong_predictions_new_names.csv", index=False)

print("Processing completed. Results saved to llama3_8bchat_individual_strong_predictions_new_names.csv")

In [17]:

# Create masks for rows where both model and human texts have or don't have names
both_have_names_mask = trimmed_df.index.isin(model_with_name.index) & trimmed_df.index.isin(human_with_name.index)
both_no_names_mask = trimmed_df.index.isin(model_without_name.index) & trimmed_df.index.isin(human_without_name.index)

# Calculate the minimum count to ensure balance
min_count = min(both_have_names_mask.sum(), both_no_names_mask.sum())

# Create the final balanced DataFrame
result_df = pd.concat([
    trimmed_df[both_have_names_mask].head(min_count),
    trimmed_df[both_no_names_mask].head(min_count)
])

# Add the name detection columns
result_df['model_text_has_name'] = ['yes'] * min_count + ['no'] * min_count
result_df['human_text_has_name'] = ['yes'] * min_count + ['no'] * min_count

# Save the result
result_df.to_csv("./llama3_8bchat_individual_strong_predictions_new_names_balanced.csv", index=False)

print(f"Processing completed. Total rows in final dataset: {len(result_df)}")
print(f"Rows with names: {min_count}, Rows without names: {min_count}")
print(f"Total columns: {result_df.shape[1]}")

Processing completed. Total rows in final dataset: 584
Rows with names: 292, Rows without names: 292
Total columns: 7


In [23]:
len(result_df[(result_df['model_text_has_name'] == 'no') & (result_df['human_text_has_name'] == 'no')])


292